In [ ]:
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 30.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import easyocr
import numpy as np
import re
from io import BytesIO
from sentence_transformers import SentenceTransformer, util
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
import cv2
from transformers import pipeline
import warnings
import logging
import os
import torch
from nltk import FreqDist
from scipy.stats import entropy as scipy_entropy
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error # Import MAE
from sklearn.preprocessing import StandardScaler, LabelEncoder
import joblib

warnings.filterwarnings('ignore')

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Download NLTK data
nltk.download('vader_lexicon', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('punkt_tab', quiet=True)

class IntelligentFeatureExtractor:
    def __init__(self, exchange_rate=88.72, input_currency='AUTO'):
        """
        exchange_rate: USD->INR rate used when converting USD prices to INR
        input_currency: 'AUTO' (infer), 'USD', or 'INR' - controls how numeric prices are interpreted
        """
        self.exchange_rate = float(exchange_rate)
        self.input_currency = input_currency
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {self.device}")
        self.reader = easyocr.Reader(['en'], gpu=torch.cuda.is_available())
        self.model = SentenceTransformer('all-MiniLM-L6-v2').to(self.device)
        self.zero_shot = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
        self.sia = SentimentIntensityAnalyzer()
        self.stop_words = set(stopwords.words('english'))
        self.generator = pipeline("text-generation", model="EleutherAI/gpt-neo-125M")
        self.known_brands = ['nabisco', 'kellogg', 'ritz', 'oreo', 'hershey', 'apple', 'samsung', 'himalaya', 'nestle']
        self.subcategory_map = {
            'teddy|graham|cracker': 'Crackers',
            'biscuit|cookie': 'Biscuits',
            'chocolate|candy': 'Chocolates',
            'snack|chip': 'Snack Foods',
            'shampoo|conditioner': 'Hair Care',
            'soap|face wash|cleanser': 'Skin Care',
            'smartphone|phone|tablet': 'Smartphones',
            'laptop|computer': 'Laptops',
            'beverage|juice|coffee|tea': 'Beverages',
            'cereal': 'Cereals',
            'toothpaste|mouthwash': 'Oral Care'
        }
        self.category_keywords = {
            'Snacks': ['teddy', 'graham', 'cracker', 'chip', 'snack', 'cookie'],
            'Personal Care': ['shampoo', 'soap', 'lotion', 'cream', 'herbal'],
            'Electronics': ['smartphone', 'laptop', 'tablet', 'device', 'charger'],
            'Beverages': ['juice', 'coffee', 'tea', 'soda', 'water'],
            'Food': ['cereal', 'pasta', 'sauce', 'spice']
        }
        self.color_map = {
            (0, 15): 'Red', (15, 30): 'Orange', (30, 60): 'Yellow',
            (60, 120): 'Green', (120, 180): 'Blue', (180, 360): 'Purple'
        }
        self.packaging_quality_map = {
            'Glass': 1.0,
            'Plastic': 0.7,
            'Paper': 0.5,
            'Cloth': 0.4,
            'Cardboard': 0.6
        }
        self.premium_colors = ['black', 'gold', 'white', 'silver', 'navy']
        self.premium_keywords = ["luxury", "premium", "organic", "limited edition", "exclusive", "heritage", "professional", "handcrafted", "high-grade", "signature", "gold-infused"]
        self.premium_origins = ["italy", "swiss", "french", "japanese"]

    def classify_zero_shot(self, text, candidates, context="Classify the product:"):
        if not text or not candidates:
            return candidates[0] if candidates else 'unknown'
        try:
            result = self.zero_shot(f"{context} {text}", candidate_labels=candidates, multi_label=False)
            return result['labels'][0]
        except Exception as e:
            logging.error(f"Error in classify_zero_shot: {e}")
            return candidates[0]

    def _is_garbage(self, text):
        words = text.split()
        if len(words) < 1:
            return True
        word_counts = FreqDist(words)
        max_freq = max(word_counts.values())
        return max_freq / len(words) > 0.5 or len(set(words)) < 2

    def _extract_from_fallback(self, fallback_text, prompt_template):
        fallback_lower = fallback_text.lower()
        if 'brand' in prompt_template.lower():
            match = re.search(r'\b([a-z]+)\b', fallback_lower)
            return match.group(1).title() if match else 'Unknown'
        elif 'category' in prompt_template.lower():
            for cat, kws in self.category_keywords.items():
                if any(kw in fallback_lower for kw in kws):
                    return cat
            return 'Unknown'
        elif 'subcategory' in prompt_template.lower():
            for pattern, subcategory in self.subcategory_map.items():
                if re.search(pattern, fallback_lower):
                    return subcategory
            return 'Unknown'
        elif 'pack quantity' in prompt_template.lower():
            match = re.search(r'(\d+)', fallback_lower)
            return int(match.group(1)) if match else 1
        elif 'weight' in prompt_template.lower():
            return 'Unknown'
        elif 'material' in prompt_template.lower():
            return 'Unknown'
        elif 'form factor' in prompt_template.lower():
            match = re.search(r'(box|bag|packs|jar|liquid|tube|device|can|bottle)', fallback_lower)
            return match.group(1).title() if match else 'Unknown'
        elif 'usage' in prompt_template.lower():
            if any(kw in fallback_lower for kw in ['phone', 'laptop', 'tablet']):
                return 'Professional'
            elif any(kw in fallback_lower for kw in ['shampoo', 'soap', 'toothpaste']):
                return 'Daily use'
            elif 'disney' in fallback_lower or any(kw in fallback_lower for kw in self.category_keywords.get('Snacks', [])):
                return 'Children’s snack'
            return 'Daily use'
        elif 'gender' in prompt_template.lower():
            return 'Unisex'
        elif 'age' in prompt_template.lower():
            return 'Children' if 'disney' in fallback_lower else 'All ages'
        elif 'color' in prompt_template.lower():
            return 'Unknown'
        elif 'certifications' in prompt_template.lower():
            return 'N/A'
        elif 'origin' in prompt_template.lower():
            return 'Unknown'
        elif 'rarity' in prompt_template.lower():
            return 'Medium'
        elif 'popularity' in prompt_template.lower():
            return 5.0
        return 'Unknown'

    def generate_classification(self, text, prompt_template, max_tokens=30, is_numeric=False, fallback_text=''):
        prompt = prompt_template.format(text=text)
        try:
            result = self.generator(prompt, max_length=len(prompt.split()) + max_tokens, num_return_sequences=1, do_sample=True, temperature=0.3, top_p=0.95)
            generated = result[0]['generated_text'].split(prompt)[-1].strip()
            clean = re.split(r'[.,!?;:\n]', generated)[0].strip()
            if len(clean) < 3 or self._is_garbage(clean) or not re.match(r'^[a-zA-Z\s\d\.\-\/]+$', clean):
                clean = self._extract_from_fallback(fallback_text, prompt_template)
            logging.info(f"Generated classification for {prompt_template}: {clean}")
            if is_numeric:
                clean_str = str(clean)
                num_match = re.search(r'(\d+(?:\.\d+)?)', clean_str)
                return float(num_match.group(1)) if num_match else 5.0
            return clean if clean else 'Unknown'
        except Exception as e:
            logging.error(f"Error in generate_classification: {e}")
            return self._extract_from_fallback(fallback_text, prompt_template)

    def load_image_from_path(self, image_path):
        try:
            color_img = cv2.imread(image_path)
            if color_img is None:
                print(f"Error loading image from path: {image_path}")
                return None, None
            img_gray = cv2.cvtColor(color_img, cv2.COLOR_BGR2GRAY)
            ocr_img = cv2.convertScaleAbs(img_gray, alpha=1.5, beta=0)
            return ocr_img, color_img
        except Exception as e:
            print(f"Error loading image from path {image_path}: {e}")
            return None, None

    def perform_ocr(self, img):
        try:
            if img is not None:
                if len(img.shape) == 2:
                    img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
                height, width = img.shape[:2]
                if width > 1000:
                    scale = 1000 / width
                    new_width = int(width * scale)
                    new_height = int(height * scale)
                    img = cv2.resize(img, (new_width, new_height))
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
                enhanced = clahe.apply(gray)
                if len(enhanced.shape) == 3:
                    enhanced = cv2.cvtColor(enhanced, cv2.COLOR_BGR2GRAY)
                _, thresh = cv2.threshold(enhanced, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
                results = self.reader.readtext(enhanced)
                ocr_original = ' '.join([text for _, text, _ in results]).strip()
                ocr_original = re.sub(r'(\w)\1{3,}', r'\1', ocr_original)
                ocr_original = ' '.join(re.sub(r'\b(\w{1,2})\s*\1+\b', r'\1', ocr_original).split())
                ocr_text = ocr_original.lower().strip()
                logging.info(f"OCR extracted text: {ocr_text[:200]}...")
                return ocr_original, ocr_text, results
            else:
                logging.warning("No image provided for OCR")
                return "", "", []
        except Exception as e:
            logging.error(f"OCR error: {e}")
            return "", "", []

    def detect_packaging_type(self, img):
        if img is None:
            return 'Unknown'
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        edges = cv2.Canny(gray, 50, 150)
        edge_density = np.sum(edges > 0) / (edges.shape[0] * edges.shape[1])
        s_mean = np.mean(hsv[:,:,1])
        v_mean = np.mean(hsv[:,:,2])
        gray_var = np.var(gray)
        if edge_density > 0.1:
            if gray_var > 2000:
                return 'Cloth'
            elif gray_var > 800:
                return 'Cardboard'
            else:
                return 'Paper'
        else:
            if s_mean < 30 and v_mean > 200:
                return 'Glass'
            else:
                return 'Plastic'
        return 'Unknown'

    def compute_glossiness_score(self, img):
        if img is None:
            return 0.0
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=5)
        sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=5)
        gradient_mag = np.sqrt(sobelx**2 + sobely**2)
        high_grad_areas = gradient_mag > np.mean(gradient_mag) + 2 * np.std(gradient_mag)
        glossiness = np.sum(high_grad_areas) / (img.shape[0] * img.shape[1])
        return glossiness

    def compute_texture_entropy(self, img):
        if img is None:
            return 0.0
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        hist = np.histogram(gray.ravel(), bins=256)[0]
        hist = hist / hist.sum() if hist.sum() > 0 else hist
        return scipy_entropy(hist) if hist.sum() > 0 else 0.0

    def compute_color_saturation_mean(self, img):
        if img is None:
            return 0.0
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        return np.mean(hsv[:,:,1])

    def compute_text_density(self, results, img_shape):
        if not results:
            return 0.0
        text_area = 0
        for detection in results:
            bbox = detection[0]
            width = max([pt[0] for pt in bbox]) - min([pt[0] for pt in bbox])
            height = max([pt[1] for pt in bbox]) - min([pt[1] for pt in bbox])
            text_area += width * height
        total_area = img_shape[0] * img_shape[1]
        return text_area / total_area if total_area > 0 else 0.0

    def compute_contrast_level(self, img):
        if img is None:
            return 0.0
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        return float(np.std(gray))

    def compute_design_simplicity(self, results, img_shape):
        text_density = self.compute_text_density(results, img_shape)
        return 1.0 - text_density

    def compute_keyword_similarity_premium(self, text):
        try:
            text_emb = self.model.encode(text)
            premium_embs = self.model.encode(self.premium_keywords)
            sims = cosine_similarity([text_emb], premium_embs)[0]
            return float(np.mean(sims))
        except Exception:
            return 0.0

    def parse_to_standard_unit(self, weight_str):
        if not weight_str or weight_str in ['N/A', 'Unknown', '', None]:
            return None, None
        s = weight_str.lower().strip()
        fl_match = re.search(r'(\d+(?:\.\d+)?)\s*(fl(?:fluid)?\s*oz|fl oz)', s)
        if fl_match:
            val = float(fl_match.group(1))
            return val * 29.5735, 'ml'
        match = re.search(r'(\d+(?:\.\d+)?)\s*(mg|g|gram|grams|kg|kilogram|ml|milliliter|l|litre|liter|oz|ounce|pound|lb|lbs)', s)
        if not match:
            return None, None
        value = float(match.group(1))
        unit = match.group(2).replace('.', '').lower()
        if unit in ('ml', 'milliliter'):
            return value, 'ml'
        if unit in ('l', 'liter', 'litre'):
            return value * 1000.0, 'ml'
        if unit in ('mg', 'milligram'):
            return value * 0.001, 'g'
        if unit in ('g', 'gram', 'grams'):
            return value, 'g'
        if unit in ('kg', 'kilogram'):
            return value * 1000.0, 'g'
        if unit in ('oz', 'ounce'):
            return value * 28.3495, 'g'
        if unit in ('lb', 'lbs', 'pound'):
            return value * 453.592, 'g'
        return None, None

    def _parse_pack_and_weight(self, text):
        text_low = (text or "").lower()
        m = re.search(r'(\d+)\s*[x×]\s*(\d+(?:\.\d+)?)\s*(fl(?:fluid)?\s*oz|oz|g|kg|ml|l|litre|liter|pound|lb|ounce|gram)', text_low)
        if m:
            pack = int(m.group(1))
            qty = float(m.group(2))
            unit = m.group(3)
            per_qty, unit_type = self.parse_to_standard_unit(f"{qty} {unit}")
            return per_qty, unit_type, pack, None, f"{qty} {unit}"
        m2 = re.search(r'(\d+(?:\.\d+)?)\s*(fl(?:fluid)?\s*oz|oz|g|kg|ml|l|litre|liter|pound|lb|ounce|gram)\s*[x×]\s*(\d+)', text_low)
        if m2:
            qty = float(m2.group(1))
            unit = m2.group(2)
            pack = int(m2.group(3))
            per_qty, unit_type = self.parse_to_standard_unit(f"{qty} {unit}")
            return per_qty, unit_type, pack, None, f"{qty} {unit}"
        m3 = re.search(r'(\d+(?:\.\d+)?)\s*(fl(?:fluid)?\s*oz|oz|g|kg|ml|l|litre|liter|pound|lb|ounce|gram)[^\d]{0,20}pack(?:\s*of)?\s*(\d+)', text_low)
        if m3:
            qty = float(m3.group(1))
            unit = m3.group(2)
            pack = int(m3.group(3))
            per_qty, unit_type = self.parse_to_standard_unit(f"{qty} {unit}")
            return per_qty, unit_type, pack, None, f"{qty} {unit}"
        m4 = re.search(r'(?:pack\s*of\s*|^|\s)(\d+)(?:\s*[-]?pack\b|\s*pack\b|\s*pcs\b|\s*ct\b)', text_low)
        if m4:
            pack = int(m4.group(1))
            m_w = re.search(r'(\d+(?:\.\d+)?)\s*(fl(?:fluid)?\s*oz|oz|g|kg|ml|l|litre|liter|pound||lb|ounce|gram)', text_low)
            if m_w:
                qty = float(m_w.group(1))
                unit = m_w.group(2)
                per_qty, unit_type = self.parse_to_standard_unit(f"{qty} {unit}")
                return per_qty, unit_type, pack, None, f"{qty} {unit}"
            return None, None, pack, None, ''
        m5 = re.search(r'(?:total|net|net wt|net weight|net volume)[:\s\-]?(\d+(?:\.\d+)?)\s(fl(?:fluid)?\s*oz|oz|g|kg|ml|l|litre|liter|pound|lb|ounce|gram)', text_low)
        if m5:
            qty = float(m5.group(1))
            unit = m5.group(2)
            total_qty, unit_type = self.parse_to_standard_unit(f"{qty} {unit}")
            return None, unit_type, 1, total_qty, f"{qty} {unit}"
        m6 = re.search(r'(\d+(?:\.\d+)?)\s*(fl(?:fluid)?\s*oz|oz|g|kg|ml|l|litre|liter|pound|lb|ounce|gram)', text_low)
        if m6:
            qty = float(m6.group(1))
            unit = m6.group(2)
            per_qty, unit_type = self.parse_to_standard_unit(f"{qty} {unit}")
            return per_qty, unit_type, 1, None, f"{qty} {unit}"
        return None, None, 1, None, ''

    def _detect_currency_and_convert_to_inr(self, price_value, catalog_text=''):
        if isinstance(price_value, str):
            s = price_value.strip()
            detected = 'UNKNOWN'
            if re.search(r'\$', s) or re.search(r'\busd\b', s, re.I):
                detected = 'USD'
            elif re.search(r'₹', s) or re.search(r'\binr\b', s, re.I):
                detected = 'INR'
            else:
                just_num = re.sub(r'[^\d\.]', '', s)
                try:
                    num = float(just_num) if just_num != '' else None
                except:
                    num = None
                if num is not None:
                    if 'usd' in catalog_text.lower() or '$' in catalog_text:
                        detected = 'USD'
                    elif 'inr' in catalog_text.lower() or '₹' in catalog_text:
                        detected = 'INR'
                    else:
                        detected = 'USD' if num < 25 else 'INR'
                else:
                    detected = 'UNKNOWN'
            numeric_part = re.sub(r'[^\d\.]', '', s)
            try:
                num = float(numeric_part)
            except:
                num = None
            if num is None:
                return None, detected
            if detected == 'USD':
                inr_price = num * self.exchange_rate
            elif detected == 'INR':
                inr_price = num
            else:
                if num < 25:
                    inr_price = num * self.exchange_rate
                    detected = 'USD(heuristic)'
                else:
                    inr_price = num
                    detected = 'INR(heuristic)'
            return float(inr_price), detected
        else:
            try:
                num = float(price_value)
            except:
                return None, 'UNKNOWN'
            if self.input_currency == 'USD':
                return float(num * self.exchange_rate), 'USD'
            if self.input_currency == 'INR':
                return float(num), 'INR'
            if 'usd' in catalog_text.lower() or '$' in catalog_text:
                return float(num * self.exchange_rate), 'USD'
            if '₹' in catalog_text or 'inr' in catalog_text.lower():
                return float(num), 'INR'
            if num < 25:
                return float(num * self.exchange_rate), 'USD(heuristic)'
            else:
                return float(num), 'INR(heuristic)'

    def extract_basic_features(self, ocr_original, ocr_text, catalog_content, price=None, img=None, results=[]):
        features = {}
        combined_original = f"{ocr_original} {catalog_content}".strip()
        combined_text = combined_original.lower()

        # Brand extraction
        brand_candidates = self.known_brands + ['gucci', 'chanel', 'rolex']
        brand_match = re.search(r'^(?:item name:\s*)?([\w&\-\']+(?:\s+[\w&\-\']+)?)', catalog_content, re.I)
        brand = brand_match.group(1).title() if brand_match else 'Unknown'
        if brand == 'Unknown':
            for b in brand_candidates:
                if b in ocr_text:
                    brand = b.title()
                    break
        if brand != 'Unknown':
            try:
                brand_emb = self.model.encode(brand.lower())
                catalog_emb = self.model.encode(catalog_content.lower())
                if cosine_similarity([brand_emb], [catalog_emb])[0][0] < 0.5:
                    brand = 'Unknown'
            except Exception:
                pass
        features['Brand'] = brand

        # Category and Subcategory
        category_candidates = list(self.category_keywords.keys())
        features['Category'] = self.classify_zero_shot(combined_text, category_candidates)
        subcategory_candidates = list(set(self.subcategory_map.values()))
        features['Subcategory'] = self.classify_zero_shot(combined_text, subcategory_candidates)

        # Pack Size
        qty_match = re.search(r'(\d+)\s*(pack|ct|pcs|units?|bundle|set|combo|pack of)', combined_text, re.I)
        features['PackSize'] = int(qty_match.group(1)) if qty_match else 1

        # Unit Measure
        uom_match_catalog = re.search(r'(fl(?:fluid)?\s*oz|oz|g|ml|l|kg|mg|lb|pound|ounce|liter|litre)', catalog_content, re.I)
        uom_match_ocr = re.search(r'(fl(?:fluid)?\s*oz|oz|g|ml|l|kg|mg|lb|pound|ounce|liter|litre)', ocr_original, re.I)
        if uom_match_catalog:
            features['UnitMeasure'] = uom_match_catalog.group(1)
        elif uom_match_ocr:
            features['UnitMeasure'] = uom_match_ocr.group(1)
        else:
            features['UnitMeasure'] = 'N/A'

        # Weight/Volume
        weight_pattern = r'(\d+(?:\.\d+)?)\s*(fl(?:fluid)?\s*oz|oz|g|kg|mg|ml|l|lb|liter|litre|pound|ounce|gram|kilogram|milliliter|fl oz)'
        weight_match = re.search(weight_pattern, ocr_original, re.I)
        if not weight_match:
            weight_match = re.search(weight_pattern, catalog_content, re.I)
        features['WeightVolume'] = weight_match.group(0).strip() if weight_match else 'Unknown'

        # Parse pack and weight
        per_item_qty, unit_type, parsed_pack_size, total_qty_from_text, parsed_raw = self._parse_pack_and_weight(combined_original)

        # Handle unit measure for beverages
        unitmeasure_norm = features['UnitMeasure'].lower() if features['UnitMeasure'] else ''
        if unitmeasure_norm in ('l', 'ml', 'fl oz', 'fl', 'fluid oz'):
            if unit_type == 'g' and re.search(r'oz|ounce', parsed_raw.lower() if parsed_raw else ''):
                m = re.search(r'(\d+(?:\.\d+)?)', parsed_raw)
                if m:
                    num = float(m.group(1))
                    per_item_qty, unit_type = num * 29.5735, 'ml'

        if parsed_pack_size and parsed_pack_size > 1:
            features['PackSize'] = parsed_pack_size

        # Compute total_quantity
        if total_qty_from_text:
            total_quantity = total_qty_from_text
            final_unit_type = unit_type
        elif per_item_qty:
            total_quantity = per_item_qty * (features['PackSize'] if features['PackSize'] > 0 else 1)
            final_unit_type = unit_type
        else:
            converted_quantity, unit_type_fallback = self.parse_to_standard_unit(features['WeightVolume'])
            if converted_quantity:
                if features['PackSize'] > 1:
                    total_quantity = converted_quantity * (features['PackSize'] if features['PackSize'] > 0 else 1)
                else:
                    total_quantity = converted_quantity
                final_unit_type = unit_type_fallback
            else:
                total_quantity = None
                final_unit_type = None

        # Price processing (only for training)
        if price is not None:
            total_price_inr, detected_currency = self._detect_currency_and_convert_to_inr(price, catalog_content + " " + ocr_original)
            if total_price_inr is None:
                features['TotalPriceINR'] = None
                features['PricePerUnit'] = 0.0
                features['PricePer100Unit'] = 0.0
                features['PriceUnit'] = 'INR/?'
                features['PriceNote'] = 'price parse failed'
            elif total_quantity is None or final_unit_type is None:
                features['TotalPriceINR'] = float(total_price_inr)
                features['PricePerUnit'] = 0.0
                features['PricePer100Unit'] = 0.0
                features['PriceUnit'] = 'INR/?'
                features['PriceNote'] = 'quantity parse failed'
            else:
                features['TotalPriceINR'] = float(total_price_inr)
                features['TotalQuantityStandard'] = float(total_quantity)
                features['UnitType'] = final_unit_type
                ppu = float(total_price_inr) / float(total_quantity)
                features['PricePerUnit'] = ppu
                features['PricePer100Unit'] = ppu * 100.0
                features['PriceUnit'] = f"INR/{'ml' if final_unit_type=='ml' else 'g'}"
                features['PriceNote'] = f"currency_detected:{detected_currency}"
        else:
            # For test set, include TotalQuantityStandard and UnitType but not price-related
            features['TotalQuantityStandard'] = float(total_quantity) if total_quantity else 0.0
            features['UnitType'] = final_unit_type if final_unit_type else 'Unknown'

        # Material
        material_candidates = ['glass', 'metal', 'thick plastic', 'thin plastic', 'cardboard', 'paper', 'cloth']
        features['Material'] = self.classify_zero_shot(combined_text, material_candidates, context="Classify the packaging material:")

        # Packaging Type
        features['PackagingType'] = self.detect_packaging_type(img)

        # Usage
        usage_candidates = ['daily use', 'professional', 'children’s snack', 'luxury']
        features['Usage'] = self.classify_zero_shot(combined_text, usage_candidates, context="Classify the usage type:")

        # Gender and Age Target
        gender_candidates = ['unisex', 'men', 'women', 'boys', 'girls']
        features['GenderTarget'] = self.classify_zero_shot(combined_text, gender_candidates, context="Classify the target gender:")
        age_candidates = ['children', 'adults', 'seniors', 'babies', 'all ages']
        features['AgeTarget'] = self.classify_zero_shot(combined_text, age_candidates, context="Classify the target age group:")

        # Color
        color_match = re.search(r'(red|blue|green|yellow|black|white|honey|golden|brown|matte|rose|silver)', combined_text, re.I)
        features['Color'] = color_match.group(1).title() if color_match else 'Unknown'

        # Certifications
        cert_match = re.search(r'(organic|gluten|kosher|vegan|natural|iso|fda|isi|ce|certified|fair trade)', combined_text, re.I)
        features['Certifications'] = cert_match.group(0).title() if cert_match else 'N/A'

        # Country of Origin
        origin_match = re.search(r'made in (\w+)', combined_text, re.I)
        origin = origin_match.group(1).title() if origin_match else 'Unknown'
        features['OriginCountry'] = origin
        features['PremiumOrigin'] = 1 if origin.lower() in self.premium_origins else 0

        # Premium Keywords
        premium_keywords_found = [kw for kw in self.premium_keywords if kw in combined_text]
        features['PremiumKeywords'] = len(premium_keywords_found)
        features['KeywordSimilarityPremium'] = self.compute_keyword_similarity_premium(combined_text)

        # Organic Keywords
        organic_keywords = re.findall(r'(organic|natural|herbal|vegan|gluten-free|eco-friendly)', combined_text, re.I)
        features['OrganicKeywords'] = len(set(organic_keywords))

        # Technical Specs
        spec_prompt = "Based on the product text: {text}\nKey technical specs or details (e.g., 8GB RAM, 130 calories) separated by commas:"
        features['TechnicalSpecs'] = self.generate_classification(combined_text, spec_prompt, max_tokens=60, fallback_text=catalog_content)

        # Discount and Charm Pricing (only for training)
        if price is not None:
            features['HasDiscount'] = 1 if re.search(r'(offer|sale|discount|win|free|buy one get|off|%)', combined_text, re.I) else 0
            features['DiscountSymbol'] = 1 if re.search(r'(\%|off)', combined_text, re.I) else 0
            features['CharmPricing'] = 1 if re.search(r'\.\d?9[5-9]', str(price)) or re.search(r'\.\d?9[5-9]', ocr_text) else 0
        else:
            features['HasDiscount'] = 0
            features['DiscountSymbol'] = 0
            features['CharmPricing'] = 0

        # Bundle
        features['IsBundle'] = 1 if features.get('PackSize', 1) > 1 or 'combo' in combined_text or 'set' in combined_text or 'bundle' in combined_text else 0

        # Barcode
        barcode_match = re.search(r'\b\d{12}\b', ocr_text)
        features['HasBarcode'] = 1 if barcode_match else 0

        # Premium Brand
        premium_brand_candidates = ['yes', 'no']
        premium_resp = self.classify_zero_shot(combined_text, premium_brand_candidates, context="Is this a premium or luxury brand?")
        features['IsPremiumBrand'] = 1 if premium_resp == 'yes' or features['PremiumKeywords'] > 0 else 0

        # Brand Popularity
        pop_prompt = "Based on the product text: {text}\nBrand popularity score on a scale of 1-10 (higher for well-known brands):"
        features['BrandPopularity'] = self.generate_classification(combined_text, pop_prompt, is_numeric=True, fallback_text=catalog_content)

        # Rarity
        rarity_candidates = ['common', 'medium', 'limited edition', 'rare']
        rarity_resp = self.classify_zero_shot(combined_text, rarity_candidates, context="Classify the rarity or novelty level:")
        rarity_map = {'common': 0.2, 'medium': 0.5, 'limited edition': 0.7, 'rare': 0.8}
        features['Rarity'] = rarity_map.get(rarity_resp.lower(), 0.5)

        # Image Features
        if img is not None:
            hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
            pixels = hsv.reshape(-1, 3)
            hue_mean = np.mean(pixels[:, 0])
            hue_std = np.std(pixels[:, 0])
            features['ColorStyle'] = 'Vibrant' if np.std(pixels[:,1]) > 50 else 'Minimalist' if np.mean(pixels[:,1]) < 50 else 'Standard'
            features['ColorHarmony'] = float(1.0 - (hue_std / 180.0))
            dominant_color = 'Standard'
            for (hue_min, hue_max), color in self.color_map.items():
                if hue_min <= hue_mean <= hue_max:
                    dominant_color = color
                    break
            features['DominantColor'] = dominant_color
            features['PremiumColorPalette'] = 1 if dominant_color.lower() in self.premium_colors else 0
            features['ImageQuality'] = float(img.shape[0] * img.shape[1] / 1e6)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            bg_variance = np.var(gray)
            features['BackgroundType'] = 'Plain' if bg_variance < 1000 else 'Cluttered'
            features['PhotographyQuality'] = 1 if bg_variance < 1000 and features['ImageQuality'] > 1 else 0
            features['GlossinessScore'] = float(self.compute_glossiness_score(img))
            features['TextureEntropy'] = float(self.compute_texture_entropy(img))
            features['ColorSaturationMean'] = float(self.compute_color_saturation_mean(img))
            features['TextDensity'] = float(self.compute_text_density(results, img.shape))
            features['ContrastLevel'] = float(self.compute_contrast_level(img))
            features['DesignSimplicity'] = float(self.compute_design_simplicity(results, img.shape))
            finish_candidates = ['glossy', 'metallic', 'matte-velvet', 'rough']
            features['FinishTexture'] = self.classify_zero_shot(combined_text, finish_candidates, context="Classify the surface finish or texture:")
            shape_candidates = ['custom', 'elegant', 'standard']
            features['ShapeGeometry'] = self.classify_zero_shot(combined_text, shape_candidates, context="Classify the shape or geometry:")
        else:
            features['ColorStyle'] = 'Standard'
            features['ColorHarmony'] = 0.5
            features['DominantColor'] = 'N/A'
            features['PremiumColorPalette'] = 0
            features['ImageQuality'] = 0
            features['BackgroundType'] = 'N/A'
            features['PhotographyQuality'] = 0
            features['GlossinessScore'] = 0.0
            features['TextureEntropy'] = 0.0
            features['ColorSaturationMean'] = 0.0
            features['TextDensity'] = 0.0
            features['ContrastLevel'] = 0.0
            features['DesignSimplicity'] = 0.5
            features['FinishTexture'] = 'Unknown'
            features['ShapeGeometry'] = 'Unknown'

        # Derived Features
        features['LogoClarity'] = float(np.mean([conf for _, _, conf in results])) if results else 0.5
        features['AuthenticityScore'] = float((features['HasBarcode'] + (1 if features['Certifications'].lower() != 'n/a' else 0) + features['LogoClarity']) / 3.0)
        features['PackagingQuality'] = self.packaging_quality_map.get(str(features['PackagingType']).title(), 0.5)

        # LuxuryIndex
        luxury_components = [
            float(features.get('BrandPopularity', 5.0)) / 10.0,
            features['PremiumKeywords'] / max(1, len(self.premium_keywords)),
            features['GlossinessScore'] if 'GlossinessScore' in features else 0.0,
            (1.0 - features['TextureEntropy'] / 10.0) if features.get('TextureEntropy', 0.0) > 0 else 0.5,
            (1.0 - features['ColorSaturationMean'] / 255.0) if 'minimalist' in str(features['ColorStyle']).lower() else 0.5,
            1.0 - features['TextDensity'] if 'TextDensity' in features else 0.5,
            features['ContrastLevel'] / 255.0 if 'ContrastLevel' in features else 0.2,
            1.0 if str(features.get('DominantColor','')).lower() in ['gold', 'black', 'silver'] else 0.5,
            features['PremiumOrigin'],
            features['IsPremiumBrand'],
            features['Rarity'],
            features['DesignSimplicity'],
            1.0 if 'glossy' in str(features.get('FinishTexture','')).lower() or 'metallic' in str(features.get('FinishTexture','')).lower() or 'matte-velvet' in str(features.get('FinishTexture','')).lower() else 0.0,
            1.0 if 'custom' in str(features.get('ShapeGeometry','')).lower() or 'elegant' in str(features.get('ShapeGeometry','')).lower() else 0.0,
            features.get('PhotographyQuality', 0)
        ]
        features['LuxuryIndex'] = float(np.mean(luxury_components))

        # Language Sophistication
        words = word_tokenize(combined_text)
        avg_word_length = np.mean([len(w) for w in words if w not in self.stop_words]) if words else 0.0
        features['LanguageSophistication'] = float(avg_word_length / 10.0) if avg_word_length > 0 else 0.0

        return features

def process_dataset(csv_path, image_folder_path, output_path, extractor_params=None, is_training=True):
    df = pd.read_csv(csv_path, encoding='latin-1')
    params = extractor_params or {}
    extractor = IntelligentFeatureExtractor(**params)
    enhanced_rows = []

    if not os.path.exists(image_folder_path):
        print(f"Error: Image folder not found at {image_folder_path}")
        return None

    image_files = sorted([f for f in os.listdir(image_folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))],
                         key=lambda x: int(re.search(r'\d+', x).group()))

    num_to_process = min(len(df), len(image_files)) # Process all available images up to df length
    if num_to_process == 0:
        print("No images or CSV rows to process.")
        return None

    for i in range(num_to_process):
        image_filename = image_files[i]
        image_path = os.path.join(image_folder_path, image_filename)
        print(f"Processing image {i+1}/{num_to_process}: {image_filename}")

        row = df.iloc[i]
        catalog_content = str(row.get('catalog_content', 'Unknown Product'))
        sample_id = row.get('sample_id', f"image_{i+1}")
        price = row.get('price', None) if is_training else None

        ocr_img, color_img = extractor.load_image_from_path(image_path)
        if color_img is None:
            print(f"Failed to load image from path: {image_path}. Skipping.")
            continue

        ocr_original, ocr_text, results = extractor.perform_ocr(ocr_img)
        features = extractor.extract_basic_features(ocr_original, ocr_text, catalog_content, price, color_img, results)

        # Preserve raw price and converted for training
        if is_training:
            try:
                total_inr = features.get('TotalPriceINR', None)
                if total_inr is None:
                    price_str = "₹0.00"
                else:
                    price_str = f"₹{float(total_inr):.2f}"
            except:
                price_str = "₹0.00"
        else:
            price_str = "N/A" # No price for test set

        features.update({
            'sample_id': sample_id, # Keep sample_id for both training and test
            'catalog_content': catalog_content,
            'image_link': image_path,
            'raw_price': price if is_training else None,
            'price': price_str
        })
        enhanced_rows.append(features)

    enhanced_df = pd.DataFrame(enhanced_rows)

    # Add business logic ordinal encodings for categoricals (to standardize and incorporate marketing strategies)
    # Usage: daily/children's low price expectation (1), professional medium (2), luxury high (3)
    usage_map = {'daily use': 1.0, "children’s snack": 1.0, 'professional': 2.0, 'luxury': 3.0}
    enhanced_df['UsageScore'] = enhanced_df['Usage'].map(usage_map).fillna(2.0)

    # AgeTarget: children/babies low affordability (1), all ages medium (2), adults/seniors higher (3)
    age_map = {'children': 1.0, 'babies': 1.0, 'all ages': 2.0, 'adults': 3.0, 'seniors': 3.0}
    enhanced_df['AgeScore'] = enhanced_df['AgeTarget'].map(age_map).fillna(2.0)

    # GenderTarget: children low (1), unisex medium (2), adults higher (3)
    gender_map = {'boys': 1.0, 'girls': 1.0, 'unisex': 2.0, 'men': 3.0, 'women': 3.0}
    enhanced_df['GenderScore'] = enhanced_df['GenderTarget'].map(gender_map).fillna(2.0)

    # For PackagingType, already have PackagingQuality as dependent numeric score

    # For WeightVolume, add numeric version for standardization (parsed value, fallback 0)
    def parse_weight_numeric(row):
        val, _ = extractor.parse_to_standard_unit(row['WeightVolume'])
        return float(val) if val is not None else 0.0
    enhanced_df['WeightVolumeNumeric'] = enhanced_df.apply(parse_weight_numeric, axis=1)

    # Drop original categoricals after creating scores (to reduce features and avoid one-hot explosion)
    enhanced_df = enhanced_df.drop(columns=['Usage', 'AgeTarget', 'GenderTarget'], errors='ignore')

    # Compute dynamic means BEFORE dropping columns (using PricePerUnit for per-unit analysis)
    if is_training and not enhanced_df.empty and 'PricePerUnit' in enhanced_df.columns:
        ppu_means = enhanced_df.groupby('PackagingType')['PricePerUnit'].mean() if 'PackagingType' in enhanced_df else pd.Series()
        for idx, row in enhanced_df.iterrows():
            enhanced_df.at[idx, 'packaging_type_mean_ppu'] = float(ppu_means.get(row.get('PackagingType', 'Unknown'), 0.0))
    elif not is_training and 'PackagingType' in enhanced_df.columns: # Ensure column exists for test data if not training
         enhanced_df['packaging_type_mean_ppu'] = 0.0 # Fill with default 0 for test set initially

    # Define columns to drop (all non-kept features EXCEPT sample_id for test)
    columns_to_drop = [
        'Material', 'OriginCountry', 'TechnicalSpecs', 'Brand', 'Category', 'Subcategory', 'UnitMeasure', 'Color',
        'Certifications', 'DiscountSymbol', 'CharmPricing', 'HasBarcode', 'Rarity', 'DominantColor', 'BackgroundType',
        'catalog_content', 'image_link', 'brand_mean_price', 'category_mean_price', 'category_frequency_rank',
        'raw_price', 'price', 'PremiumKeywords', 'PremiumOrigin', 'IsPremiumBrand', 'BrandPopularity', 'ColorStyle',
        'PremiumColorPalette', 'PhotographyQuality', 'ColorSaturationMean', 'TextDensity', 'ContrastLevel',
        'DesignSimplicity', 'FinishTexture', 'ShapeGeometry', 'LogoClarity', 'AuthenticityScore', 'LuxuryIndex',
        'LanguageSophistication', 'IsBundle'
    ]
    if is_training:
        columns_to_drop.extend(['PricePerUnit', 'PricePer100Unit', 'PriceUnit', 'PriceNote'])

    # Always keep sample_id in the final DataFrame regardless of is_training
    columns_to_keep_always = ['sample_id']
    final_columns = [col for col in enhanced_df.columns if col not in columns_to_drop or col in columns_to_keep_always]
    enhanced_df = enhanced_df[final_columns]


    # For training, drop rows with invalid TotalPriceINR
    if is_training:
        enhanced_df = enhanced_df.dropna(subset=['TotalPriceINR'])
        enhanced_df = enhanced_df[enhanced_df['TotalPriceINR'] > 0]

    if not enhanced_df.empty:
        enhanced_df.to_excel(output_path, index=False)
        print(f"Features extracted to {output_path}")
        # print(enhanced_df.T.to_string()) # Suppress detailed output for training set
    else:
        print("No features extracted. Ensure images are in the specified folder and filenames match.")

    return enhanced_df

# Function to calculate SMAPE
def smape(y_true, y_pred):
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    # Handle cases where denominator is zero
    # Avoid division by zero by replacing 0s in denominator with a small epsilon
    denominator[denominator == 0] = np.finfo(float).eps
    return np.mean(numerator / denominator) * 100


def train_and_predict(train_csv_path, test_csv_path, train_image_folder, test_image_folder, train_output_path, test_output_path, model_save_path, extractor_params=None):
    exchange_rate = extractor_params.get('exchange_rate', 88.72) if extractor_params else 88.72

    # Process training dataset
    print("Processing training dataset...")
    train_df = process_dataset(train_csv_path, train_image_folder, train_output_path, extractor_params, is_training=True)

    if train_df is None or train_df.empty:
        print("Training dataset processing failed or empty.")
        return None, None, None

    # Define kept features (excluding price-dependent for prediction)
    kept_features = [
        'PackSize', 'WeightVolume', 'TotalQuantityStandard', 'PackagingType', 'PackagingQuality',
        'KeywordSimilarityPremium', 'OrganicKeywords', 'HasDiscount', 'ColorHarmony', 'ImageQuality',
        'GlossinessScore', 'TextureEntropy', 'UnitType', 'UsageScore', 'GenderScore', 'AgeScore',
        'WeightVolumeNumeric', 'packaging_type_mean_ppu' # Include the mean PPU feature
    ]

    # Filter features
    available_kept = [col for col in kept_features if col in train_df.columns]
    X_train = train_df[available_kept].fillna(0.0)
    y_train = train_df['TotalPriceINR']

    # Encode and scale
    cat_cols = ['PackagingType', 'WeightVolume', 'UnitType']
    num_cols = [col for col in X_train.columns if col not in cat_cols]
    le_dict = {}
    X_train_encoded = X_train.copy()
    for col in cat_cols:
        if col in X_train.columns:
            le = LabelEncoder()
            X_train_encoded[col] = le.fit_transform(X_train[col].astype(str))
            le_dict[col] = le

    scaler = StandardScaler()
    X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_encoded), columns=X_train_encoded.columns, index=X_train_encoded.index)

    # Train model
    print("\nTraining Random Forest Regressor...")
    model = RandomForestRegressor(n_estimators=30, max_depth=3, min_samples_split=4, min_samples_leaf=2, random_state=42)
    model.fit(X_train_scaled, y_train)

    # Save model, scaler, encoders
    joblib.dump(model, model_save_path)
    joblib.dump(scaler, model_save_path.replace('.joblib', '_scaler.joblib'))
    joblib.dump(le_dict, model_save_path.replace('.joblib', '_encoders.joblib'))
    print(f"\nTrained model saved to {model_save_path}")

    # Process test dataset
    print("\nProcessing test dataset...")
    # Ensure sample_id is kept in the processed test_df
    test_df = process_dataset(test_csv_path, test_image_folder, test_output_path, extractor_params, is_training=False)


    if test_df is None or test_df.empty:
        print("Test dataset processing failed or empty.")
        return train_df, None, None

    # Ensure test set has the same features and handle 'packaging_type_mean_ppu' for test set
    # We need to load the means from the training data to apply to the test data consistently
    if 'packaging_type_mean_ppu' not in test_df.columns:
         test_df['packaging_type_mean_ppu'] = 0.0 # Initialize if not present

    available_kept_test = [col for col in kept_features if col in test_df.columns]
    X_test = test_df[available_kept_test].fillna(0.0)
    X_test_encoded = X_test.copy()

    # Apply label encoding using the training encoders
    for col in cat_cols:
        if col in X_test.columns:
            le = le_dict.get(col)
            if le:
                # Handle unseen labels by mapping to a default class (e.g., the first class)
                # Ensure the default class is a string before passing to transform
                default_class = le.classes_[0] if le.classes_.size > 0 else 'Unknown' # Handle empty classes case
                X_test_encoded[col] = X_test[col].astype(str).map(lambda x: x if x in le.classes_ else default_class)
                X_test_encoded[col] = le.transform(X_test_encoded[col].astype(str))
            else:
                # If no encoder found for a categorical column (shouldn't happen if kept_features are consistent),
                # fill with a default value or handle as appropriate
                X_test_encoded[col] = 0 # Or another default


    # Apply standardization using the training scaler
    X_test_scaled = pd.DataFrame(scaler.transform(X_test_encoded), columns=X_test_encoded.columns, index=X_test_encoded.index)


    # Predict on test set
    print("\nGenerating predictions for test set...")
    y_pred_test = model.predict(X_test_scaled)
    y_pred_test_usd = y_pred_test / exchange_rate

    # Save predictions to CSV
    test_results = pd.DataFrame({
        'sample_id': test_df['sample_id'],
        'predicted_price_usd': y_pred_test_usd
    })
    test_results_path = test_output_path.replace('.xlsx', '_predictions.csv')
    test_results.to_csv(test_results_path, index=False)
    print(f"\nTest predictions saved to {test_results_path}")

    # **Explicitly state that test metrics cannot be calculated without actual test prices**
    print("\nTest Evaluation Metrics (Cannot be calculated without actual prices in test.csv):")
    print("MSE (USD): N/A")
    print("R2 (USD): N/A")
    print("SMAPE (USD): N/A%")
    print("MAE (USD): N/A")
    print("RMSE (USD): N/A")

    # Display only the test set predictions
    print("\nTest Set Predictions (USD):")
    display(test_results)

    return train_df, test_df, test_predictions

if __name__ == "__main__":
    train_csv_path = '/content/drive/MyDrive/AMAZON-ML/training_dataset_3.csv'
    test_csv_path = '/content/drive/MyDrive/AMAZON-ML/test.csv'
    train_image_folder = '/content/drive/MyDrive/AMAZON-ML/Images1'
    test_image_folder = '/content/drive/MyDrive/AMAZON-ML/test_images'
    train_output_path = '/content/drive/MyDrive/AMAZON-ML/train_output_amazon.xlsx'
    test_output_path = '/content/drive/MyDrive/AMAZON-ML/test_output_amazon.xlsx'
    model_save_path = '/content/drive/MyDrive/AMAZON-ML/random_forest_model_final.joblib'
    extractor_params = {'exchange_rate': 88.72, 'input_currency': 'AUTO'}

    train_df, test_df, test_predictions = train_and_predict(
        train_csv_path, test_csv_path, train_image_folder, test_image_folder,
        train_output_path, test_output_path, model_save_path, extractor_params
    )

In [ ]:
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 28.4 MB/s eta 0:00:00
